<a href="https://colab.research.google.com/github/Akith-002/Daredevils_tech-triathlon_datathon/blob/main/Tech_triathlon_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [ ]:
places_df = pd.read_excel("/content/drive/MyDrive/csv files/Tech-Triathlon - Datathon Datasets/Places Dataset.xlsx")
visitors_df = pd.read_excel("/content/drive/MyDrive/csv files/Tech-Triathlon - Datathon Datasets/Visitors Preference Dataset.xlsx")

In [ ]:
places_df.head()

,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
0,Arugam Bay Beach,6.840408,81.836848,"Arugam Bay Beach, Sri Lanka",4.8,1591.0,['Arugam Bay Beach is a surfer's paradise! I s...
1,Mirissa Beach,5.944703,80.459161,"Mirissa, Sri Lanka",4.6,1748.0,['Mirissa Beach is truly a gem on Sri LankaÃ¢Â...
2,Weligama Beach (surf and stay),5.972486,80.435714,"Weligama, Sri Lanka",4.4,325.0,['Weligama Beach is a fantastic spot for both ...
3,Ahangama,5.973975,80.362159,"Ahangama, Sri Lanka",NaN,NaN,['Ahangama was a bit disappointing for me as a...
4,Hikkaduwa Beach,6.137727,80.099060,"Hikkaduwa Beach, Sri Lanka",4.7,1438.0,['Hikkaduwa Beach is a delightful escape for s...


- added an index column

In [ ]:
places_df['place_id'] = places_df.index
# Make the 'places id' column the first column
columns = ['place_id'] + [col for col in places_df.columns if col != 'place_id']
places_df = places_df[columns]

# Verify the updated column order
places_df.head()

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
0,0,Arugam Bay Beach,6.840408,81.836848,"Arugam Bay Beach, Sri Lanka",4.8,1591.0,['Arugam Bay Beach is a surfer's paradise! I s...
1,1,Mirissa Beach,5.944703,80.459161,"Mirissa, Sri Lanka",4.6,1748.0,['Mirissa Beach is truly a gem on Sri LankaÃ¢Â...
2,2,Weligama Beach (surf and stay),5.972486,80.435714,"Weligama, Sri Lanka",4.4,325.0,['Weligama Beach is a fantastic spot for both ...
3,3,Ahangama,5.973975,80.362159,"Ahangama, Sri Lanka",NaN,NaN,['Ahangama was a bit disappointing for me as a...
4,4,Hikkaduwa Beach,6.137727,80.099060,"Hikkaduwa Beach, Sri Lanka",4.7,1438.0,['Hikkaduwa Beach is a delightful escape for s...


In [ ]:
places_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411 entries, 0 to 410
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   place_id            411 non-null    int64  
 1   name                411 non-null    object 
 2   lat                 410 non-null    float64
 3   lng                 410 non-null    float64
 4   formatted_address   411 non-null    object 
 5   rating              355 non-null    float64
 6   user_ratings_total  355 non-null    float64
 7   latest_reviews      411 non-null    object 
dtypes: float64(4), int64(1), object(3)
memory usage: 25.8+ KB


In [ ]:
places_df.describe()

,place_id,lat,lng,rating,user_ratings_total
count,411.00000,410.000000,410.000000,355.000000,355.000000
mean,205.00000,7.304668,80.583211,4.459437,1608.639437
std,118.78973,0.994841,0.518571,0.433211,3607.678752
min,0.00000,5.941381,79.694183,0.900000,27.000000
25%,102.50000,6.629356,80.213274,4.300000,147.000000
50%,205.00000,7.012780,80.536523,4.500000,375.000000
75%,307.50000,7.947930,80.946773,4.750000,1245.500000
max,410.00000,9.820859,81.859583,5.000000,26736.000000


In [ ]:
places_df.isnull().sum()

,0
place_id,0
name,0
lat,1
lng,1
formatted_address,0
rating,56
user_ratings_total,56
latest_reviews,0


In [ ]:
places_df['name'].nunique()

398

In [ ]:
duplicate_place_names = places_df[places_df.duplicated(subset='name', keep=False)]

grouped_duplicates = duplicate_place_names.groupby('name')

for name, group in grouped_duplicates:
    print(f"\nRows with similar Name '{name}':\n")
    print(group)


Rows with similar Name 'Ambuluwawa Biodiversity Complex':

     place_id                             name       lat        lng  \
104       104  Ambuluwawa Biodiversity Complex  7.161627  80.547197   
398       398  Ambuluwawa Biodiversity Complex  7.161629  80.547189   

       formatted_address  rating  user_ratings_total  \
104  Gampola , Sri Lanka     4.5              3248.0   
398   Gampola, Sri Lanka     4.7              3254.0   

                                        latest_reviews  
104  ['Ambuluwawa Biodiversity Complex is a hidden ...  
398  ['The Ambuluwawa Biodiversity Complex is a hid...  

Rows with similar Name 'Bundala National Park':

     place_id                   name       lat        lng  \
19         19  Bundala National Park  6.199086  81.210493   
405       405  Bundala National Park  6.193822  81.187854   

        formatted_address  rating  user_ratings_total  \
19    bundala , Sri Lanka     4.3               654.0   
405  Weligatta, Sri Lanka     4.3     

- cant drop duplicated rows because they have different reviews
- you can combine the reviews put them in one row
- take mean values of rating,lat,lng put them in one row

---
###handling 104 and 398 place_id

In [ ]:
# Get the reviews for both IDs
reviews_104 = places_df.loc[places_df['place_id'] == 104, 'latest_reviews'].values[0]
reviews_398 = places_df.loc[places_df['place_id'] == 398, 'latest_reviews'].values[0]

# Combine the reviews
combined_reviews = reviews_104 + reviews_398

places_df.loc[places_df['place_id'] == 104, 'latest_reviews'] = combined_reviews
places_df.loc[places_df['place_id'] == 104, 'latest_reviews'].values[0]

"['Ambuluwawa Biodiversity Complex is a hidden gem! The climb up the tower is thrilling, and though narrow, it adds to the adventure. The panoramic views from the top are simply breathtaking, especially with mist rolling in. I recommend going early to avoid crowds. The coffee shop at the base is a nice spot to recharge afterward. Perfect for anyone looking to connect with nature!' , 'What a unique experience! The biodiversity complex offers stunning landscapes, and the tower climb is definitely not for the faint-hearted. I loved the peaceful atmosphere, but I wish there were more shaded areas along the path. We took a tuk-tuk up, which saved us some energy for the climb. Overall, a must-visit, especially for photography enthusiasts!' , 'We visited Ambuluwawa as a family, and it was quite an adventure! While the ascent to the tower was challenging, the kids loved it. The views at the top are worth every step, but I recommend being cautious if you're not great with heights. The adventure

In [ ]:
# Get the row indices where places id is 104 and 398
row_104 = places_df[places_df['place_id'] == 104].index[0]
row_398 = places_df[places_df['place_id'] == 398].index[0]

# Calculate the mean for the 'rating', 'lat', and 'lng' columns
mean_rating = places_df.loc[[row_104, row_398], 'rating'].mean()
mean_lat = places_df.loc[[row_104, row_398], 'lat'].mean()
mean_lng = places_df.loc[[row_104, row_398], 'lng'].mean()

# Assign the mean values to the row with places id = 104
places_df.loc[row_104, ['rating', 'lat', 'lng']] = [mean_rating, mean_lat, mean_lng]

# Verify the result
places_df.loc[row_104, ['rating', 'lat', 'lng']]

,104
rating,4.6
lat,7.161628
lng,80.547193


In [ ]:
# Get the row indices for the places with places id 104 and 398
row_104 = places_df[places_df['place_id'] == 104].index[0]
row_398 = places_df[places_df['place_id'] == 398].index[0]

# Copy the formatted_address from the row with places id 398 to the row with places id 104
places_df.at[row_104, "formatted_address"] = places_df.at[row_398, "formatted_address"]

# Verify the result
places_df.at[row_104, "formatted_address"]

'Gampola, Sri Lanka'

In [ ]:
# Calculate the sum of 'user_ratings_total' for place_id 104 and 398
user_ratings_sum = places_df.loc[places_df['place_id'].isin([104, 398]), 'user_ratings_total'].sum()

# Assign the sum to 'user_ratings_total' for place_id 104
places_df.loc[places_df['place_id'] == 104, 'user_ratings_total'] = user_ratings_sum

# Verify the result
places_df.loc[places_df['place_id'] == 104, 'user_ratings_total']

,user_ratings_total
104,6502.0


In [ ]:
# Drop the row where place_id is 398
places_df = places_df[places_df['place_id'] != 398].reset_index(drop=True)

In [ ]:
places_df[places_df['place_id'] == 398]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews


---
###handling 19 and 405 place_id

In [ ]:
# print rows with place_id 19 and 405
places_df[places_df['place_id'].isin([19, 405])]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
19,19,Bundala National Park,6.199086,81.210493,"bundala , Sri Lanka",4.3,654.0,['Bundala National Park is a hidden gem for na...
404,405,Bundala National Park,6.193822,81.187854,"Weligatta, Sri Lanka",4.3,561.0,['Bundala National Park is a hidden gem! I arr...


In [ ]:
# Get the reviews for both IDs
reviews_19 = places_df.loc[places_df['place_id'] == 19, 'latest_reviews'].values[0]
reviews_405 = places_df.loc[places_df['place_id'] == 405, 'latest_reviews'].values[0]

# Combine the reviews
combined_reviews = reviews_19 + reviews_405

places_df.loc[places_df['place_id'] == 19, 'latest_reviews'] = combined_reviews
places_df.loc[places_df['place_id'] == 19, 'latest_reviews'].values[0]

'[\'Bundala National Park is a hidden gem for nature lovers! The variety of birds is simply amazing; I was fortunate to see both native and migratory species. The early morning safari is a must for the best sightings and the beautiful sunrise. Our guide was knowledgeable and passionate about the wildlife, making the experience even more enjoyable. I would recommend bringing a good camera to capture the breathtaking scenery and wildlife encounters. Overall, a fantastic outing for bird watchers and photographers alike!\',\n\'Visiting Bundala National Park was a highlight of my trip to Sri Lanka. The tranquil environment and the chance to see elephants and a multitude of birds made for an unforgettable experience. We booked a half-day safari, and our guide was very helpful, pointing out various species and explaining their behaviors. However, I wish the park had more facilities for visitors. It could use some better signage and rest areas. Still, the beauty of nature here far outweighs an

In [ ]:
user_ratings_sum = places_df.loc[places_df['place_id'].isin([19, 405]), 'user_ratings_total'].sum()

places_df.loc[places_df['place_id'] == 19, 'user_ratings_total'] = user_ratings_sum

# Verify the result
places_df.loc[places_df['place_id'] == 19, 'user_ratings_total']

,user_ratings_total
19,1215.0


In [ ]:
row_19 = places_df[places_df['place_id'] == 19].index[0]
row_405 = places_df[places_df['place_id'] == 405].index[0]

# Calculate the mean for the 'rating', 'lat', and 'lng' columns
mean_rating = places_df.loc[[row_19, row_405], 'rating'].mean()
mean_lat = places_df.loc[[row_19, row_405], 'lat'].mean()
mean_lng = places_df.loc[[row_19, row_405], 'lng'].mean()

places_df.loc[row_19, ['rating', 'lat', 'lng']] = [mean_rating, mean_lat, mean_lng]

# Verify the result
places_df.loc[row_19, ['rating', 'lat', 'lng']]

,19
rating,4.3
lat,6.196454
lng,81.199174


In [ ]:
row_19 = places_df[places_df['place_id'] == 19].index[0]
row_405 = places_df[places_df['place_id'] == 405].index[0]

# Copy the formatted_address from the row with places id 398 to the row with places id 104
places_df.at[row_19, "formatted_address"] = places_df.at[row_405, "formatted_address"]

# Verify the result
places_df.at[row_19, "formatted_address"]

'Weligatta, Sri Lanka'

In [ ]:
places_df = places_df[places_df['place_id'] != 405].reset_index(drop=True)
places_df[places_df['place_id'] == 405]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews


---
### handling 52 and 391 place_id

In [ ]:
places_df[places_df['place_id'].isin([52, 391])]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
52,52,Coral Sanctuary Boat Ticket Issue Center,6.137342,80.099253,ÃƒÂ Ã‚Â·Ã‚Â„ÃƒÂ Ã‚Â·Ã‚Â’ÃƒÂ Ã‚Â¶Ã‚ÂšÃƒÂ Ã‚Â·Ã‚...,4.1,209.0,['I had a fantastic time at the Coral Sanctuar...
391,391,Coral Sanctuary Boat Ticket Issue Center,6.137358,80.099252,"Hikkaduwa, Sri Lanka",4.4,131.0,['The Coral Sanctuary Boat Ticket Issue Center...


In [ ]:
# Get the reviews for both IDs
reviews_52 = places_df.loc[places_df['place_id'] == 52, 'latest_reviews'].values[0]
reviews_391 = places_df.loc[places_df['place_id'] == 391, 'latest_reviews'].values[0]

# Combine the reviews
combined_reviews = reviews_52 + reviews_391

places_df.loc[places_df['place_id'] == 52, 'latest_reviews'] = combined_reviews
places_df.loc[places_df['place_id'] == 52, 'latest_reviews'].values[0]

"['I had a fantastic time at the Coral Sanctuary! The snorkeling was beautiful, and I was thrilled to see a variety of fish and a few turtles. The staff were friendly and knowledgeable, ensuring we had all the gear we needed. Just be cautious of brokers trying to sell you more expensive boat trips. Overall, a memorable experience that I highly recommend!','While the snorkeling was decent, I was a bit disappointed with the coral qualityÃ¢Â€Â”it seems to have diminished over the years. The staff were great, though, and the boat ride was enjoyable. If youÃ¢Â€Â™re looking for a relaxing day on the water, this is still a good choice, but manage your expectations about the coral.','The Coral Sanctuary was a delightful adventure for our family! The kids loved snorkeling and spotting colorful fish. The staff made sure everyone was safe and having fun. Just beware of parking scams nearby; itÃ¢Â€Â™s best to go straight to the ticket center. A great way to spend an afternoon!','If you come here, 

In [ ]:
user_ratings_sum = places_df.loc[places_df['place_id'].isin([52, 391]), 'user_ratings_total'].sum()

places_df.loc[places_df['place_id'] == 52, 'user_ratings_total'] = user_ratings_sum

# Verify the result
places_df.loc[places_df['place_id'] == 52, 'user_ratings_total']

,user_ratings_total
52,340.0


In [ ]:
row_52 = places_df[places_df['place_id'] == 52].index[0]
row_391 = places_df[places_df['place_id'] == 391].index[0]

# Calculate the mean for the 'rating', 'lat', and 'lng' columns
mean_rating = places_df.loc[[row_52, row_391], 'rating'].mean()
mean_lat = places_df.loc[[row_52, row_391], 'lat'].mean()
mean_lng = places_df.loc[[row_52, row_391], 'lng'].mean()

places_df.loc[row_52, ['rating', 'lat', 'lng']] = [mean_rating, mean_lat, mean_lng]

# Verify the result
places_df.loc[row_52, ['rating', 'lat', 'lng']]

,52
rating,4.25
lat,6.13735
lng,80.099253


In [ ]:
row_52 = places_df[places_df['place_id'] == 52].index[0]
row_391 = places_df[places_df['place_id'] == 391].index[0]

# Copy the formatted_address from the row with places id 398 to the row with places id 104
places_df.at[row_52, "formatted_address"] = places_df.at[row_391, "formatted_address"]

# Verify the result
places_df.at[row_52, "formatted_address"]

'Hikkaduwa, Sri Lanka'

In [ ]:
places_df = places_df[places_df['place_id'] != 391].reset_index(drop=True)
places_df[places_df['place_id'] == 391]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews


---
### handling 22 and 395 place_id

In [ ]:
places_df[places_df['place_id'].isin([22, 395])]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
22,22,Dambulla Royal Cave Temple and Golden Temple,7.854914,80.65057,"Dambulla, Sri Lanka",4.2,11598.0,['The Dambulla Royal Cave Temple is a breathta...
394,395,Dambulla Royal Cave Temple and Golden Temple,7.854914,80.65057,"Dambulla, Sri Lanka",4.3,11690.0,['Dambulla Royal Cave Temple is a must-visit f...


In [ ]:
# Get the reviews for both IDs
reviews_22 = places_df.loc[places_df['place_id'] == 22, 'latest_reviews'].values[0]
reviews_395 = places_df.loc[places_df['place_id'] == 395, 'latest_reviews'].values[0]

# Combine the reviews
combined_reviews = reviews_22 + reviews_395

places_df.loc[places_df['place_id'] == 22, 'latest_reviews'] = combined_reviews
places_df.loc[places_df['place_id'] == 22, 'latest_reviews'].values[0]

"['The Dambulla Royal Cave Temple is a breathtaking experience! The climb is steep, but the views and the stunning cave paintings make it worthwhile. I was fascinated by the intricate details of the Buddha statues. Just a tip: wear comfortable shoes and bring water, as it gets hot! The atmosphere is serene, perfect for some spiritual reflection. Definitely a must-visit if youÃ¢Â€Â™re in Sri Lanka.', 'Visiting the Golden Temple and the caves was the highlight of our trip! The architecture is simply stunning, and the cultural significance of the site is palpable. The climb was challenging, especially in the heat, but the beauty of the surroundings kept us going. Be mindful of the monkeysÃ¢Â€Â”theyÃ¢Â€Â™re quite cheeky! Overall, a fantastic blend of culture and spirituality.', 'As a solo traveler, I found the Dambulla Cave Temple to be an enriching experience. The art inside the caves is mesmerizing, and I loved taking pictures of the intricate sculptures. However, be prepared for a lot o

In [ ]:
user_ratings_sum = places_df.loc[places_df['place_id'].isin([22, 395]), 'user_ratings_total'].sum()

places_df.loc[places_df['place_id'] == 22, 'user_ratings_total'] = user_ratings_sum

# Verify the result
places_df.loc[places_df['place_id'] == 22, 'user_ratings_total']

,user_ratings_total
22,23288.0


In [ ]:
row_22 = places_df[places_df['place_id'] == 22].index[0]
row_395 = places_df[places_df['place_id'] == 395].index[0]

# Calculate the mean for the 'rating', 'lat', and 'lng' columns
mean_rating = places_df.loc[[row_22, row_395], 'rating'].mean()
mean_lat = places_df.loc[[row_22, row_395], 'lat'].mean()
mean_lng = places_df.loc[[row_22, row_395], 'lng'].mean()

places_df.loc[row_22, ['rating', 'lat', 'lng']] = [mean_rating, mean_lat, mean_lng]

# Verify the result
places_df.loc[row_22, ['rating', 'lat', 'lng']]

,22
rating,4.25
lat,7.854914
lng,80.65057


In [ ]:
# Copy the formatted_address from the row with places id 398 to the row with places id 104
places_df.at[row_22, "formatted_address"] = places_df.at[row_395, "formatted_address"]

# Verify the result
places_df.at[row_22, "formatted_address"]

'Dambulla, Sri Lanka'

In [ ]:
places_df = places_df[places_df['place_id'] != 395].reset_index(drop=True)
places_df[places_df['place_id'] == 395]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews


---
### handling 127 and 400 place_id

In [ ]:
places_df[places_df['place_id'].isin([127, 400])]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
127,127,Elephant Transit Home,6.426689,80.816536,"Udawalawa, Sri Lanka",4.6,1849.0,['Visiting the Elephant Transit Home was a hea...
397,400,Elephant Transit Home,6.426689,80.816536,"Udawalawa, Sri Lanka",4.6,1784.0,['The Elephant Transit Home is a heartwarming ...


In [ ]:
# Get the reviews for both IDs
reviews_127 = places_df.loc[places_df['place_id'] == 127, 'latest_reviews'].values[0]
reviews_400 = places_df.loc[places_df['place_id'] == 400, 'latest_reviews'].values[0]

# Combine the reviews
combined_reviews = reviews_127 + reviews_400

places_df.loc[places_df['place_id'] == 127, 'latest_reviews'] = combined_reviews
places_df.loc[places_df['place_id'] == 127, 'latest_reviews'].values[0]

"['Visiting the Elephant Transit Home was a heartwarming experience! Watching the baby elephants being fed was adorable and truly educational. The staff clearly cares for these animals, which makes it a meaningful visit. However, I wish there were more information provided during the visit about the elephantsÃ¢Â€Â™ stories. Overall, a must-see if youÃ¢Â€Â™re in Sri Lanka!' , 'What a delightful experience! Seeing the baby elephants up close during feeding time was a highlight of our trip. The facility does a fantastic job of rehabilitating these little ones. The only downside was the lack of organization at the entrance; it felt a bit chaotic. Still, I left feeling inspired by their conservation efforts.' , 'We visited the Elephant Transit Home as a family and were enchanted by the playful baby elephants. It was incredible to see them being fed and cared for by the dedicated staff. However, I expected a bit more interactive experiences for kids. Nonetheless, itÃ¢Â€Â™s a worthwhile stop 

In [ ]:
user_ratings_sum = places_df.loc[places_df['place_id'].isin([127, 400]), 'user_ratings_total'].sum()

places_df.loc[places_df['place_id'] == 127, 'user_ratings_total'] = user_ratings_sum

# Verify the result
places_df.loc[places_df['place_id'] == 127, 'user_ratings_total']

,user_ratings_total
127,3633.0


In [ ]:
row_127 = places_df[places_df['place_id'] == 127].index[0]
row_400 = places_df[places_df['place_id'] == 400].index[0]

# Calculate the mean for the 'rating', 'lat', and 'lng' columns
mean_rating = places_df.loc[[row_127, row_400], 'rating'].mean()
mean_lat = places_df.loc[[row_127, row_400], 'lat'].mean()
mean_lng = places_df.loc[[row_127, row_400], 'lng'].mean()

places_df.loc[row_127, ['rating', 'lat', 'lng']] = [mean_rating, mean_lat, mean_lng]

# Verify the result
places_df.loc[row_127, ['rating', 'lat', 'lng']]

,127
rating,4.6
lat,6.426689
lng,80.816536


In [ ]:
# Copy the formatted_address from the row with places id 398 to the row with places id 104
places_df.at[row_127, "formatted_address"] = places_df.at[row_400, "formatted_address"]

# Verify the result
places_df.at[row_127, "formatted_address"]

'Udawalawa, Sri Lanka'

In [ ]:
places_df = places_df[places_df['place_id'] != 400].reset_index(drop=True)
places_df[places_df['place_id'] == 400]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews


---
### handling 134 and 403 place_id


In [ ]:
places_df[places_df['place_id'].isin([134, 403])]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
134,134,Kanneliya National Rain Forest Reserve | ÃƒÂ Ã...,6.257642,80.359621,"Sri, Sri Lanka",4.8,172.0,['Kanneliya National Rain Forest Reserve is a ...
399,403,Kanneliya National Rain Forest Reserve | ÃƒÂ Ã...,6.257642,80.359621,"Sri, Sri Lanka",4.7,143.0,['Kanneliya National Rain Forest Reserve is a ...


In [ ]:
# Get the reviews for both IDs
reviews_134 = places_df.loc[places_df['place_id'] == 134, 'latest_reviews'].values[0]
reviews_403 = places_df.loc[places_df['place_id'] == 403, 'latest_reviews'].values[0]

# Combine the reviews
combined_reviews = reviews_134 + reviews_403

places_df.loc[places_df['place_id'] == 134, 'latest_reviews'] = combined_reviews
places_df.loc[places_df['place_id'] == 134, 'latest_reviews'].values[0]

"['Kanneliya National Rain Forest Reserve is a hidden gem! The hiking trails are well-maintained, and the biodiversity is astounding. Our guide was knowledgeable and made the experience even more enjoyable with his stories. The waterfalls were breathtaking, perfect for a refreshing dip. Just remember to arrive early, as they close by 3 PM. Overall, a must-visit for nature lovers!','I visited Kanneliya with a group of friends, and we had a blast! The lush greenery and unique wildlife made our hike unforgettable. We spent hours exploring, but I wish we had planned for more time to relax by the waterfalls. Just be cautious of the slippery paths. Definitely a highlight of our trip to Sri Lanka!','As a family, we had a wonderful day at Kanneliya. The kids loved spotting different plants and insects, and the guide was fantastic with them. The natural swimming area was a hit, but I do wish there were more facilities for families. Nonetheless, a beautiful place to connect with nature and enjoy

In [ ]:
user_ratings_sum = places_df.loc[places_df['place_id'].isin([134, 403]), 'user_ratings_total'].sum()

places_df.loc[places_df['place_id'] == 134, 'user_ratings_total'] = user_ratings_sum

# Verify the result
places_df.loc[places_df['place_id'] == 134, 'user_ratings_total']

,user_ratings_total
134,315.0


In [ ]:
row_134 = places_df[places_df['place_id'] == 134].index[0]
row_403 = places_df[places_df['place_id'] == 403].index[0]

# Calculate the mean for the 'rating', 'lat', and 'lng' columns
mean_rating = places_df.loc[[row_134, row_403], 'rating'].mean()
mean_lat = places_df.loc[[row_134, row_403], 'lat'].mean()
mean_lng = places_df.loc[[row_134, row_403], 'lng'].mean()

places_df.loc[row_134, ['rating', 'lat', 'lng']] = [mean_rating, mean_lat, mean_lng]

# Verify the result
places_df.loc[row_134, ['rating', 'lat', 'lng']]

,134
rating,4.75
lat,6.257642
lng,80.359621


In [ ]:
# Copy the formatted_address from the row with places id 398 to the row with places id 104
places_df.at[row_134, "formatted_address"] = places_df.at[row_403, "formatted_address"] = "Kanneliya, Sri Lanka"

# Verify the result
places_df.at[row_134, "formatted_address"]

'Kanneliya, Sri Lanka'

In [ ]:
import re

problematic_text = places_df.at[row_134, "name"]

# Use regex to extract only the part before the unreadable characters
cleaned_text = re.match(r'^[^\|]+', problematic_text).group(0)

places_df.at[row_134, "name"] = cleaned_text
places_df.at[row_134, "name"]

'Kanneliya National Rain Forest Reserve '

In [ ]:
places_df = places_df[places_df['place_id'] != 403].reset_index(drop=True)
places_df[places_df['place_id'] == 403]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews


---
### handling 102 and 397 place_id

In [ ]:
places_df[places_df['place_id'].isin([102, 397])]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
102,102,Kataragama,6.413546,81.332568,"Kataragama, Sri Lanka",NaN,NaN,"['Kataragama has a rich cultural significance,..."
395,397,Kataragama,6.413546,81.332568,"Kataragama, Sri Lanka",NaN,NaN,['Kataragama was a bit of a letdown for me. Th...


#### how to handle NaN values in the rating ???

In [ ]:
# Get the reviews for both IDs
reviews_102 = places_df.loc[places_df['place_id'] == 102, 'latest_reviews'].values[0]
reviews_397 = places_df.loc[places_df['place_id'] == 397, 'latest_reviews'].values[0]

# Combine the reviews
combined_reviews = reviews_102 + reviews_397

places_df.loc[places_df['place_id'] == 102, 'latest_reviews'] = combined_reviews
places_df.loc[places_df['place_id'] == 102, 'latest_reviews'].values[0]

"['Kataragama has a rich cultural significance, but I found the crowds overwhelming, especially during festivals. The temples are beautiful, but the lack of clear signage made it difficult to appreciate the history behind them. ItÃ¢Â€Â™s worth visiting if youÃ¢Â€Â™re interested in spirituality, but be prepared for chaos.' , 'I was excited to explore Kataragama, but it felt more commercialized than I expected. The shops and stalls were a distraction from the spiritual atmosphere. While the temples are interesting, I didnÃ¢Â€Â™t find the experience very engaging. I recommend visiting if youÃ¢Â€Â™re nearby, but donÃ¢Â€Â™t make a special trip.' , 'Visiting Kataragama was a mixed bag. The temples are stunning, and the rituals are fascinating to observe, but the heat and lack of shade made it uncomfortable at times. ItÃ¢Â€Â™s a place of worship, so respect is essential, but I wish there were more facilities for tourists to enjoy the visit.' , 'Kataragama is a significant pilgrimage site, but

In [ ]:
row_102 = places_df[places_df['place_id'] == 102].index[0]
row_397 = places_df[places_df['place_id'] == 397].index[0]

# Calculate the mean for the 'rating', 'lat', and 'lng' columns
mean_lat = places_df.loc[[row_102, row_397], 'lat'].mean()
mean_lng = places_df.loc[[row_102, row_397], 'lng'].mean()

places_df.loc[row_102, ['lat', 'lng']] = [mean_lat, mean_lng]

# Verify the result
places_df.loc[row_102, ['lat', 'lng']]

,102
lat,6.413546
lng,81.332568


In [ ]:
# Copy the formatted_address from the row with places id 398 to the row with places id 104
places_df.at[row_102, "formatted_address"] = places_df.at[row_397, "formatted_address"]

# Verify the result
places_df.at[row_102, "formatted_address"]

'Kataragama, Sri Lanka'

In [ ]:
places_df = places_df[places_df['place_id'] != 397].reset_index(drop=True)
places_df[places_df['place_id'] == 397]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews


---
### handling 84 and 394 place_id

In [ ]:
places_df[places_df['place_id'].isin([84, 394])]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
84,84,Limestone Caves|ÃƒÂ Ã‚Â®Ã‚ÂšÃƒÂ Ã‚Â¯Ã‚Â_x0081_...,9.820859,80.144544,"Valvettithurai, Sri Lanka",4.4,196.0,['The Limestone Caves are a hidden gem in Jaff...
393,394,Limestone Caves|ÃƒÂ Ã‚Â®Ã‚ÂšÃƒÂ Ã‚Â¯Ã‚Â_x0081_...,9.820859,80.144544,"Valvettithurai, Sri Lanka",4.1,177.0,['The Limestone Caves are truly a hidden gem! ...


In [ ]:
# Get the reviews for both IDs
reviews_84 = places_df.loc[places_df['place_id'] == 84, 'latest_reviews'].values[0]
reviews_394 = places_df.loc[places_df['place_id'] == 394, 'latest_reviews'].values[0]

# Combine the reviews
combined_reviews = reviews_134 + reviews_403

places_df.loc[places_df['place_id'] == 84, 'latest_reviews'] = combined_reviews
places_df.loc[places_df['place_id'] == 84, 'latest_reviews'].values[0]

"['Kanneliya National Rain Forest Reserve is a hidden gem! The hiking trails are well-maintained, and the biodiversity is astounding. Our guide was knowledgeable and made the experience even more enjoyable with his stories. The waterfalls were breathtaking, perfect for a refreshing dip. Just remember to arrive early, as they close by 3 PM. Overall, a must-visit for nature lovers!','I visited Kanneliya with a group of friends, and we had a blast! The lush greenery and unique wildlife made our hike unforgettable. We spent hours exploring, but I wish we had planned for more time to relax by the waterfalls. Just be cautious of the slippery paths. Definitely a highlight of our trip to Sri Lanka!','As a family, we had a wonderful day at Kanneliya. The kids loved spotting different plants and insects, and the guide was fantastic with them. The natural swimming area was a hit, but I do wish there were more facilities for families. Nonetheless, a beautiful place to connect with nature and enjoy

In [ ]:
user_ratings_sum = places_df.loc[places_df['place_id'].isin([84, 394]), 'user_ratings_total'].sum()

places_df.loc[places_df['place_id'] == 84, 'user_ratings_total'] = user_ratings_sum

# Verify the result
places_df.loc[places_df['place_id'] == 84, 'user_ratings_total']

,user_ratings_total
84,373.0


In [ ]:
row_84 = places_df[places_df['place_id'] == 84].index[0]
row_394 = places_df[places_df['place_id'] == 394].index[0]

# Calculate the mean for the 'rating', 'lat', and 'lng' columns
mean_rating = places_df.loc[[row_84, row_394], 'rating'].mean()
mean_lat = places_df.loc[[row_84, row_394], 'lat'].mean()
mean_lng = places_df.loc[[row_84, row_394], 'lng'].mean()

places_df.loc[row_84, ['rating', 'lat', 'lng']] = [mean_rating, mean_lat, mean_lng]

# Verify the result
places_df.loc[row_84, ['rating', 'lat', 'lng']]

,84
rating,4.25
lat,9.820859
lng,80.144544


In [ ]:
# Copy the formatted_address from the row with places id 398 to the row with places id 104
places_df.at[row_84, "formatted_address"] = places_df.at[row_394, "formatted_address"]

# Verify the result
places_df.at[row_84, "formatted_address"]

'Valvettithurai, Sri Lanka'

In [ ]:
import re

problematic_text = places_df.at[row_84, "name"]

# Use regex to extract only the part before the unreadable characters
cleaned_text = re.match(r'^[^\|]+', problematic_text).group(0)

places_df.at[row_84, "name"] = cleaned_text
places_df.at[row_84, "name"]

'Limestone Caves'

In [ ]:
places_df = places_df[places_df['place_id'] != 394].reset_index(drop=True)
places_df[places_df['place_id'] == 394]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews


---
### handling 124 and 399 place_id

In [ ]:
places_df[places_df['place_id'].isin([124, 399])]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
124,124,Madunagala Hot Water Spring,6.25363,80.981711,Sri Lanka,4.6,1117.0,['Madunagala Hot Water Spring is a hidden gem!...
394,399,Madunagala Hot Water Spring,6.25363,80.981711,Sri Lanka,4.5,1151.0,['Madunagala Hot Water Spring is a hidden gem!...


In [ ]:
# Get the reviews for both IDs
reviews_124 = places_df.loc[places_df['place_id'] == 124, 'latest_reviews'].values[0]
reviews_399 = places_df.loc[places_df['place_id'] == 399, 'latest_reviews'].values[0]

# Combine the reviews
combined_reviews = reviews_124 + reviews_399

places_df.loc[places_df['place_id'] == 124, 'latest_reviews'] = combined_reviews
places_df.loc[places_df['place_id'] == 124, 'latest_reviews'].values[0]

"['Madunagala Hot Water Spring is a hidden gem! The mineral-rich waters are incredibly relaxing, and the surrounding scenery is beautiful. I spent hours soaking in the warm pools. However, the aquarium and animal park could use some love and maintenance. ItÃ¢Â€Â™s a great place to unwind, but I hope they improve the facilities soon.', 'Visiting Madunagala was a delightful experience! The hot springs were clean and well-maintained, perfect for a relaxing soak. The ticket prices are quite reasonable, especially for locals. I enjoyed the serene environment, although the kidsÃ¢Â€Â™ play area could use some upkeep. Overall, a lovely spot for a day trip.', 'What a unique experience at Madunagala! The hot water springs were delightful, providing a perfect respite from the heat. The gardens were nice, but I noticed that the aquarium needs more attention. ItÃ¢Â€Â™s a shame, as it could add to the experience. Still, a worthwhile visit for anyone looking to relax and enjoy nature.', 'I had a love

In [ ]:
user_ratings_sum = places_df.loc[places_df['place_id'].isin([124, 399]), 'user_ratings_total'].sum()

places_df.loc[places_df['place_id'] == 124, 'user_ratings_total'] = user_ratings_sum

# Verify the result
places_df.loc[places_df['place_id'] == 124, 'user_ratings_total']

,user_ratings_total
124,2268.0


In [ ]:
row_124 = places_df[places_df['place_id'] == 124].index[0]
row_399 = places_df[places_df['place_id'] == 399].index[0]

# Calculate the mean for the 'rating', 'lat', and 'lng' columns
mean_rating = places_df.loc[[row_124, row_399], 'rating'].mean()
mean_lat = places_df.loc[[row_124, row_399], 'lat'].mean()
mean_lng = places_df.loc[[row_124, row_399], 'lng'].mean()

places_df.loc[row_124, ['rating', 'lat', 'lng']] = [mean_rating, mean_lat, mean_lng]

# Verify the result
places_df.loc[row_124, ['rating', 'lat', 'lng']]

,124
rating,4.55
lat,6.25363
lng,80.981711


In [ ]:
# Copy the formatted_address from the row with places id 398 to the row with places id 104
places_df.at[row_124, "formatted_address"] = places_df.at[row_399, "formatted_address"]

# Verify the result
places_df.at[row_124, "formatted_address"]

'Sri Lanka'

In [ ]:
places_df = places_df[places_df['place_id'] != 399].reset_index(drop=True)
places_df[places_df['place_id'] == 399]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews


---
### handling 70 and 392 place_id

In [ ]:
places_df[places_df['place_id'].isin([70, 392])]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
70,70,Makandawa Conservation Center (Makandawa Forest),6.99092,80.405368,"Kitulgala, Sri Lanka",4.9,127.0,['Makandawa Conservation Center is a true gem!...
391,392,Makandawa Conservation Center (Makandawa Forest),6.99092,80.405368,"Kitulgala, Sri Lanka",4.5,192.0,['Makandawa Conservation Center is a hidden ge...


In [ ]:
# Get the reviews for both IDs
reviews_70 = places_df.loc[places_df['place_id'] == 70, 'latest_reviews'].values[0]
reviews_392 = places_df.loc[places_df['place_id'] == 392, 'latest_reviews'].values[0]

# Combine the reviews
combined_reviews = reviews_70 + reviews_392

places_df.loc[places_df['place_id'] == 70, 'latest_reviews'] = combined_reviews
places_df.loc[places_df['place_id'] == 70, 'latest_reviews'].values[0]

"['Makandawa Conservation Center is a true gem! The biodiversity here is astonishing, and the boat ride to the forest entrance was a highlight. We spotted several endemic species and enjoyed the refreshing swim at Ã¢Â€Âœatha vatuna vala.Ã¢Â€Â_x009d_ Just be prepared for leeches and bring some insect repellent! Overall, a fantastic way to connect with nature and unwind.', 'I visited Makandawa with my friends and had a blast! The lush greenery and peaceful atmosphere provided the perfect escape from the city. We also tried river rafting, which was exhilarating! One tip: make sure to check the ferry prices beforehand, as we felt a bit overcharged. Still, we loved the adventure and the stunning natural beauty!', 'As a couple, we thoroughly enjoyed our time at Makandawa. The serene surroundings and the unique biodiversity made for a romantic getaway. The boat ride was a charming experience, and we appreciated the guidance from the staff. However, the entrance process was a bit confusing, so

In [ ]:
user_ratings_sum = places_df.loc[places_df['place_id'].isin([70, 392]), 'user_ratings_total'].sum()

places_df.loc[places_df['place_id'] == 70, 'user_ratings_total'] = user_ratings_sum

# Verify the result
places_df.loc[places_df['place_id'] == 70, 'user_ratings_total']

,user_ratings_total
70,319.0


In [ ]:
row_70 = places_df[places_df['place_id'] == 70].index[0]
row_392 = places_df[places_df['place_id'] == 392].index[0]

# Calculate the mean for the 'rating', 'lat', and 'lng' columns
mean_rating = places_df.loc[[row_70, row_392], 'rating'].mean()
mean_lat = places_df.loc[[row_70, row_392], 'lat'].mean()
mean_lng = places_df.loc[[row_70, row_392], 'lng'].mean()

places_df.loc[row_70, ['rating', 'lat', 'lng']] = [mean_rating, mean_lat, mean_lng]

# Verify the result
places_df.loc[row_70, ['rating', 'lat', 'lng']]

,70
rating,4.7
lat,6.99092
lng,80.405368


In [ ]:
places_df.at[row_70, "formatted_address"] = places_df.at[row_392, "formatted_address"]

# Verify the result
places_df.at[row_70, "formatted_address"]

'Kitulgala, Sri Lanka'

In [ ]:
places_df = places_df[places_df['place_id'] != 392].reset_index(drop=True)
places_df[places_df['place_id'] == 392]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews


---
### handling 6 and 410 place_id

In [ ]:
places_df[places_df['place_id'].isin([6, 410])]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
6,6,Unawatuna Beach,6.009686,80.248424,"Unawatuna Beach, Sri Lanka",4.8,1868.0,['Unawatuna Beach is a slice of paradise! The ...
400,410,Unawatuna Beach,6.009686,80.248424,"Unawatuna Beach, Sri Lanka",4.3,1908.0,['Unawatuna Beach is a paradise! The soft sand...


In [ ]:
# Get the reviews for both IDs
reviews_6 = places_df.loc[places_df['place_id'] == 6, 'latest_reviews'].values[0]
reviews_410 = places_df.loc[places_df['place_id'] == 410, 'latest_reviews'].values[0]

# Combine the reviews
combined_reviews = reviews_6 + reviews_410

places_df.loc[places_df['place_id'] == 6, 'latest_reviews'] = combined_reviews
places_df.loc[places_df['place_id'] == 6, 'latest_reviews'].values[0]

"['Unawatuna Beach is a slice of paradise! The water is crystal clear, perfect for snorkeling. I was amazed by the vibrant marine life just a few feet from the shore. The beach itself can get crowded, but it adds to the lively atmosphere. There are plenty of beach shacks to grab a bite, and the sunset views are breathtaking. A must-visit if youÃ¢Â€Â™re in Sri Lanka!', 'I had a great time at Unawatuna Beach with my friends. Paddleboarding was a highlight, and we loved exploring the coastline. While the beach is stunning, it can feel a bit too touristy at times. Still, the overall vibe is fun, and the local food stalls offer delicious snacks. Definitely worth a visit!', 'As a couple, we enjoyed our time at Unawatuna Beach, but it was more crowded than we expected. The snorkeling spots were fantastic, and we even saw some sea turtles! However, finding a quiet spot to relax was a bit challenging. If youÃ¢Â€Â™re looking for a lively beach atmosphere, this is the place for you!', 'We visited

In [ ]:
user_ratings_sum = places_df.loc[places_df['place_id'].isin([6, 410]), 'user_ratings_total'].sum()

places_df.loc[places_df['place_id'] == 6, 'user_ratings_total'] = user_ratings_sum

# Verify the result
places_df.loc[places_df['place_id'] == 6, 'user_ratings_total']

,user_ratings_total
6,3776.0


In [ ]:
row_6 = places_df[places_df['place_id'] == 6].index[0]
row_410 = places_df[places_df['place_id'] == 410].index[0]

# Calculate the mean for the 'rating', 'lat', and 'lng' columns
mean_rating = places_df.loc[[row_6, row_410], 'rating'].mean()
mean_lat = places_df.loc[[row_6, row_410], 'lat'].mean()
mean_lng = places_df.loc[[row_6, row_410], 'lng'].mean()

places_df.loc[row_6, ['rating', 'lat', 'lng']] = [mean_rating, mean_lat, mean_lng]

# Verify the result
places_df.loc[row_6, ['rating', 'lat', 'lng']]

,6
rating,4.55
lat,6.009686
lng,80.248424


In [ ]:
places_df.at[row_6, "formatted_address"] = places_df.at[row_410, "formatted_address"]

# Verify the result
places_df.at[row_6, "formatted_address"]

'Unawatuna Beach, Sri Lanka'

In [ ]:
places_df = places_df[places_df['place_id'] != 410].reset_index(drop=True)
places_df[places_df['place_id'] == 410]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews


---
### handling 31 and 402 place_id

In [ ]:
places_df[places_df['place_id'].isin([31, 402])]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
31,31,Ventura Beach,6.423471,79.99532,"Ventura Beach, Sri Lanka",4.4,516.0,"['Ventura Beach is a hidden gem! The soft, whi..."
394,402,Ventura Beach,6.423471,79.99532,"Ventura Beach, Sri Lanka",4.4,502.0,['Ventura Beach is a hidden gem! The golden sa...


In [ ]:
# Get the reviews for both IDs
reviews_31 = places_df.loc[places_df['place_id'] == 31, 'latest_reviews'].values[0]
reviews_402 = places_df.loc[places_df['place_id'] == 402, 'latest_reviews'].values[0]

# Combine the reviews
combined_reviews = reviews_31 + reviews_402

places_df.loc[places_df['place_id'] == 31, 'latest_reviews'] = combined_reviews
places_df.loc[places_df['place_id'] == 31, 'latest_reviews'].values[0]

"['Ventura Beach is a hidden gem! The soft, white sand is perfect for lounging, and the water is refreshing. I spent my days swimming and trying out wind-surfing. The locals are friendly, and while the sunbeds can be a bit pricey, itÃ¢Â€Â™s worth it for the experience. Just be sure to bring your own snacks as options are limited. Overall, a lovely spot to unwind!', 'We visited Ventura Beach as a family and had a fantastic time! The kids loved playing in the waves, and we all tried scuba diving. The instructors were professional and made it enjoyable for everyone. The beach itself is stunning, though be prepared for some hassle from sunbed sellers. Still, it was a memorable day, and we would definitely return!', 'As a solo traveler, I found Ventura Beach to be a peaceful escape. The sunsets here are breathtaking! I enjoyed bird watching early in the morning and was lucky enough to spot a few exotic species. Just a tip: the public toilets could use some improvement. Otherwise, a great pl

In [ ]:
user_ratings_sum = places_df.loc[places_df['place_id'].isin([31, 402]), 'user_ratings_total'].sum()

places_df.loc[places_df['place_id'] == 31, 'user_ratings_total'] = user_ratings_sum

# Verify the result
places_df.loc[places_df['place_id'] == 31, 'user_ratings_total']

,user_ratings_total
31,1018.0


In [ ]:
row_31 = places_df[places_df['place_id'] == 31].index[0]
row_402 = places_df[places_df['place_id'] == 402].index[0]

# Calculate the mean for the 'rating', 'lat', and 'lng' columns
mean_rating = places_df.loc[[row_31, row_402], 'rating'].mean()
mean_lat = places_df.loc[[row_31, row_402], 'lat'].mean()
mean_lng = places_df.loc[[row_31, row_402], 'lng'].mean()

places_df.loc[row_31, ['rating', 'lat', 'lng']] = [mean_rating, mean_lat, mean_lng]

# Verify the result
places_df.loc[row_31, ['rating', 'lat', 'lng']]

,31
rating,4.4
lat,6.423471
lng,79.99532


In [ ]:
places_df.at[row_31, "formatted_address"] = places_df.at[row_402, "formatted_address"]

# Verify the result
places_df.at[row_31, "formatted_address"]

'Ventura Beach, Sri Lanka'

In [ ]:
places_df = places_df[places_df['place_id'] != 402].reset_index(drop=True)
places_df[places_df['place_id'] == 402]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews


---
### handling 130 and 401 place_id

In [ ]:
places_df[places_df['place_id'].isin([130, 401])]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
130,130,Victor Hasselblad Sea Turtle Research And Cons...,6.333705,80.027906,"Kosgoda, Sri Lanka",4.3,1449.0,['The Victor Hasselblad Sea Turtle Research an...
393,401,Victor Hasselblad Sea Turtle Research And Cons...,6.333735,80.027903,"Kosgoda, Sri Lanka",4.3,1499.0,['This center is a gem! The commitment to turt...


In [ ]:
# Get the reviews for both IDs
reviews_130 = places_df.loc[places_df['place_id'] == 130, 'latest_reviews'].values[0]
reviews_401 = places_df.loc[places_df['place_id'] == 401, 'latest_reviews'].values[0]

# Combine the reviews
combined_reviews = reviews_130 + reviews_401

places_df.loc[places_df['place_id'] == 130, 'latest_reviews'] = combined_reviews
places_df.loc[places_df['place_id'] == 130, 'latest_reviews'].values[0]

"['The Victor Hasselblad Sea Turtle Research and Conservation Centre is a gem! I loved learning about the conservation efforts and getting up close with the turtles. The staff is incredibly passionate, and their dedication shines through. It was a bit crowded when I visited, but it didnÃ¢Â€Â™t take away from the experience. Definitely a place that should be on every travelerÃ¢Â€Â™s list in Sri Lanka!', 'Visiting the turtle center was a highlight of my trip. The facility is clean and well-organized, and the staff shared fascinating insights about each turtle species. I had the chance to hold a baby turtle, which was such a memorable experience. However, I wish there were more interactive activities available for visitors. Still, a great educational stop!', 'What a wonderful experience! The dedication of the staff at the Victor Hasselblad Centre is commendable, and itÃ¢Â€Â™s evident that they genuinely care for the turtles. I appreciated the guided tour, which provided in-depth informati

In [ ]:
user_ratings_sum = places_df.loc[places_df['place_id'].isin([130, 401]), 'user_ratings_total'].sum()

places_df.loc[places_df['place_id'] == 130, 'user_ratings_total'] = user_ratings_sum

# Verify the result
places_df.loc[places_df['place_id'] == 130, 'user_ratings_total']

,user_ratings_total
130,2948.0


In [ ]:
row_130 = places_df[places_df['place_id'] == 130].index[0]
row_401 = places_df[places_df['place_id'] == 401].index[0]

# Calculate the mean for the 'rating', 'lat', and 'lng' columns
mean_rating = places_df.loc[[row_130, row_401], 'rating'].mean()
mean_lat = places_df.loc[[row_130, row_401], 'lat'].mean()
mean_lng = places_df.loc[[row_130, row_401], 'lng'].mean()

places_df.loc[row_130, ['rating', 'lat', 'lng']] = [mean_rating, mean_lat, mean_lng]

# Verify the result
places_df.loc[row_130, ['rating', 'lat', 'lng']]

,130
rating,4.3
lat,6.33372
lng,80.027905


In [ ]:
places_df.at[row_130, "formatted_address"] = places_df.at[row_401, "formatted_address"]

# Verify the result
places_df.at[row_130, "formatted_address"]

'Kosgoda, Sri Lanka'

In [ ]:
places_df = places_df[places_df['place_id'] != 401].reset_index(drop=True)
places_df[places_df['place_id'] == 401]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews


---
## removing special characters from names and addresses

In [ ]:
# Define a function to detect non-ASCII characters in the 'name' column
def has_non_ascii(name):
    return any(ord(char) > 127 for char in name)

# Apply the function to find rows with non-ASCII characters in the 'name' column
non_ascii_rows = places_df[places_df['name'].apply(has_non_ascii)]

# Display the rows with problematic characters
print(non_ascii_rows)

     place_id                                               name       lat  \
99         99           ÃƒÂ„Ã‚Â€riapÃƒÂ„Ã‚Â_x0081_la Mask Museum  6.241272   
105       105  Velgam Vehera Buddhist Temple | ÃƒÂ Ã‚Â·Ã‚Â€Ãƒ...  8.647187   
107       107  Spa Ceylon Heritage Spa & Boutique II ÃƒÂ¢Ã‚Â€...  6.026257   
136       136  Buddhangala Raja Maha Viharaya ÃƒÂ Ã‚Â¶Ã‚Â¶ÃƒÂ...  7.331836   
143       143  Archaeological Museum Mihintale - ÃƒÂ Ã‚Â¶Ã‚Â¸...  8.355528   
145       145  Tantirimale Rajamaha Viharaya - ÃƒÂ Ã‚Â¶Ã‚Â­Ãƒ...  8.573602   
159       159  Corbett's Gap ÃƒÂ Ã‚Â¶Ã‚Â…ÃƒÂ Ã‚Â¶Ã‚Â§ÃƒÂ Ã‚Â·...  7.372765   
160       160  Dematamal Viharaya| ÃƒÂ Ã‚Â¶Ã‚Â¯ÃƒÂ Ã‚Â·Ã‚Â™Ãƒ...  6.753990   
161       161  Galabedda Biso Pond ÃƒÂ Ã‚Â¶Ã‚ÂœÃƒÂ Ã‚Â¶Ã‚Â½Ãƒ...  6.916830   
166       166  Batticaloa Gate | ÃƒÂ Ã‚Â®Ã‚Â®ÃƒÂ Ã‚Â®Ã‚ÂŸÃƒÂ ...  7.713625   
170       170  Kotte Rajamaha Vihara ÃƒÂ Ã‚Â¶Ã‚ÂšÃƒÂ Ã‚Â·Ã‚Â_...  6.881008   
181       181  SrÃƒÂ„Ã‚Â« VijayÃƒÂ„Ã‚Â_x0081_rÃƒÂ„Ã‚Â_x0081_m...

In [ ]:
places_df[places_df['formatted_address'].apply(has_non_ascii)]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
173,173,Mayan Water Park,7.068894,79.902236,ÃƒÂ Ã‚Â¶Ã‚Â¢ÃƒÂ Ã‚Â·Ã‚Â_x008f_-ÃƒÂ Ã‚Â¶Ã‚Â‡ÃƒÂ...,3.7,562.0,['Mayan Water Park was a decent outing for my ...
195,195,Fort Entrance (Old Gate),6.028114,80.218447,ÃƒÂ Ã‚Â¶Ã‚Â¸ÃƒÂ Ã‚Â·Ã‚Â_x008f_ÃƒÂ Ã‚Â¶Ã‚Â­ÃƒÂ ...,4.8,231.0,"['The Fort Entrance, or Old Gate, is a captiva..."
209,209,"Nilavarai, Natural underground water well | Ãƒ...",9.754379,80.088261,Urumpirai NorthÃƒÂ˜Ã‚ÂŒ Sri Lanka,4.4,1105.0,['Visiting Nilavarai Well was a highlight of m...
286,286,Kudawa Beach Kalpitiya,8.227305,79.728156,ÃƒÂ Ã‚Â¶Ã‚Â´ÃƒÂ Ã‚Â·Ã‚Â”ÃƒÂ Ã‚Â¶Ã‚Â­ÃƒÂ Ã‚Â·Ã‚...,4.3,329.0,['Kudawa Beach Kalpitiya is a hidden gem! The ...
363,363,ÃƒÂ Ã‚Â¶Ã‚Â”ÃƒÂ Ã‚Â¶Ã‚ÂšÃƒÂ Ã‚Â¶Ã‚Â±ÃƒÂ Ã‚Â·Ã‚...,6.627940,81.770818,ÃƒÂ Ã‚Â¶Ã‚Â…ÃƒÂ Ã‚Â¶Ã‚Â¸ÃƒÂ Ã‚Â·Ã‚ÂŠÃƒÂ Ã‚Â¶Ã‚...,4.5,84.0,['Okanda Beach is a hidden gem! The serene atm...


In [ ]:
import re

# Function to clean the names by removing unreadable characters
def clean_name(name):
    # Use a regular expression to remove non-ASCII characters
    cleaned_name = re.sub(r'[^\x00-\x7F]+', '', name)
    # Optionally remove specific patterns like '_x0081_' or '| '
    cleaned_name = re.sub(r'_x[0-9a-fA-F]+_', '', cleaned_name)
    cleaned_name = cleaned_name.replace('| ', '').strip()
    return cleaned_name

# Apply the cleaning function to the 'name' and 'formatted_address' columns
places_df['name'] = places_df['name'].apply(clean_name)
places_df['formatted_address'] = places_df['formatted_address'].apply(clean_name)

In [ ]:
places_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   place_id            398 non-null    int64  
 1   name                398 non-null    object 
 2   lat                 397 non-null    float64
 3   lng                 397 non-null    float64
 4   formatted_address   398 non-null    object 
 5   rating              343 non-null    float64
 6   user_ratings_total  343 non-null    float64
 7   latest_reviews      398 non-null    object 
dtypes: float64(4), int64(1), object(3)
memory usage: 25.0+ KB


In [ ]:
places_df[places_df['lat'].isnull()]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
89,89,Leisure World,NaN,NaN,Sri Lanka,NaN,NaN,"['Leisure World has potential, but my experien..."


In [ ]:
# Update the DataFrame from the internet
places_df.at[89, 'lat'] = 6.9216
places_df.at[89, 'lng'] = 80.1056

---
## removing special characters from the latest reviews

In [ ]:
places_df.loc[places_df['place_id'] == 1, 'latest_reviews'].values[0]

"['Mirissa Beach is truly a gem on Sri LankaÃ¢Â€Â™s southern coast! The soft, golden sand and crystal-clear waters are perfect for lounging or taking a dip. I went whale watching and was lucky enough to see several majestic creatures! The beachfront dining options are fantastic; I enjoyed fresh seafood while watching the sunset. However, it can get a bit crowded during peak times, but the vibrant atmosphere is worth it. Definitely a must-visit!' , 'What an amazing beach! Mirissa offers fantastic surfing conditions, though I would recommend it for more experienced surfers due to the strong waves. I also explored some local cafes and found great options for beachfront dining. The sunset views are breathtaking! On the downside, the beach can get quite busy, which sometimes detracts from the serene vibe. Overall, a great place for an adventurous day out!' , 'This beach exceeded my expectations! The snorkeling was a highlight; I saw so many colorful fish and beautiful corals. I also enjoyed

In [ ]:
def clean_text(text):
    # Replace special characters with proper ones or remove unwanted characters
    text = re.sub(r'Ã¢Â€Â™', "'", text)  # Replace the encoded apostrophe
    text = re.sub(r'[^a-zA-Z0-9.,!?\'\s]', '', text)  # Remove other unwanted special characters
    return text

def process_reviews(row):
    reviews = row['latest_reviews']  # Access the reviews
    cleaned_reviews = [clean_text(review) for review in reviews]  # Clean each review
    combined_string = ''.join(cleaned_reviews)
    return combined_string

In [ ]:
places_df['cleaned_reviews'] = places_df.apply(process_reviews, axis=1)
places_df.head()

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews,cleaned_reviews
0,0,Arugam Bay Beach,6.840408,81.836848,"Arugam Bay Beach, Sri Lanka",4.8,1591.0,['Arugam Bay Beach is a surfer's paradise! I s...,'Arugam Bay Beach is a surfer's paradise! I sp...
1,1,Mirissa Beach,5.944703,80.459161,"Mirissa, Sri Lanka",4.6,1748.0,['Mirissa Beach is truly a gem on Sri LankaÃ¢Â...,'Mirissa Beach is truly a gem on Sri Lankas so...
2,2,Weligama Beach (surf and stay),5.972486,80.435714,"Weligama, Sri Lanka",4.4,325.0,['Weligama Beach is a fantastic spot for both ...,'Weligama Beach is a fantastic spot for both b...
3,3,Ahangama,5.973975,80.362159,"Ahangama, Sri Lanka",NaN,NaN,['Ahangama was a bit disappointing for me as a...,'Ahangama was a bit disappointing for me as a ...
4,4,Hikkaduwa Beach,6.137727,80.099060,"Hikkaduwa Beach, Sri Lanka",4.7,1438.0,['Hikkaduwa Beach is a delightful escape for s...,'Hikkaduwa Beach is a delightful escape for so...


In [ ]:
places_df[places_df['rating'].isna()]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews,cleaned_reviews
3,3,Ahangama,5.973975,80.362159,"Ahangama, Sri Lanka",NaN,NaN,['Ahangama was a bit disappointing for me as a...,'Ahangama was a bit disappointing for me as a ...
5,5,Tangalle,6.024338,80.794073,"Tangalle, Sri Lanka",NaN,NaN,['Tangalle was a bit of a letdown for me. The ...,'Tangalle was a bit of a letdown for me. The b...
20,20,Anawilundawa Wetland,7.709595,79.821270,"Anawilundawa Wetland, Sri Lanka",NaN,NaN,['I visited Anawilundawa Wetland expecting a v...,'I visited Anawilundawa Wetland expecting a vi...
23,23,Anuradhapura,8.311352,80.403651,"Anuradhapura, Sri Lanka",NaN,NaN,"['Anuradhapura is rich in history, but I found...","'Anuradhapura is rich in history, but I found ..."
34,34,Uppuveli,8.601562,81.219524,"Trincomalee, Sri Lanka",NaN,NaN,"['Uppuveli has a stunning shoreline, but I fou...","'Uppuveli has a stunning shoreline, but I foun..."
35,35,Koggala,6.000704,80.335236,"Koggala, Sri Lanka",NaN,NaN,"['Koggala beach has stunning scenery, but I wa...","'Koggala beach has stunning scenery, but I was..."
37,37,Marakolliya,6.061372,80.818401,"Marakolliya, Sri Lanka",NaN,NaN,"['Marakolliya has stunning scenery, but my exp...","'Marakolliya has stunning scenery, but my expe..."
46,46,Mirissa,5.948262,80.471587,"Mirissa, Sri Lanka",NaN,NaN,"['I recently went on a sea cruise in Mirissa, ...","'I recently went on a sea cruise in Mirissa, a..."
47,47,Port City Colombo,6.937804,79.836843,"Port City Colombo, Sri Lanka",NaN,NaN,"['Port City Colombo has a lot of potential, bu...","'Port City Colombo has a lot of potential, but..."
50,50,Pitawala Nature Trail,7.548983,80.751473,"Pitawala Nature Trail, Sri Lanka",NaN,NaN,['The Pitawala Nature Trail was a bit underwhe...,'The Pitawala Nature Trail was a bit underwhel...


In [ ]:
places_df[places_df['formatted_address'].str.contains("Sri Lanka", na=False)]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews,cleaned_reviews
0,0,Arugam Bay Beach,6.840408,81.836848,"Arugam Bay Beach, Sri Lanka",4.8,1591.0,['Arugam Bay Beach is a surfer's paradise! I s...,'Arugam Bay Beach is a surfer's paradise! I sp...
1,1,Mirissa Beach,5.944703,80.459161,"Mirissa, Sri Lanka",4.6,1748.0,['Mirissa Beach is truly a gem on Sri LankaÃ¢Â...,'Mirissa Beach is truly a gem on Sri Lankas so...
2,2,Weligama Beach (surf and stay),5.972486,80.435714,"Weligama, Sri Lanka",4.4,325.0,['Weligama Beach is a fantastic spot for both ...,'Weligama Beach is a fantastic spot for both b...
3,3,Ahangama,5.973975,80.362159,"Ahangama, Sri Lanka",NaN,NaN,['Ahangama was a bit disappointing for me as a...,'Ahangama was a bit disappointing for me as a ...
4,4,Hikkaduwa Beach,6.137727,80.099060,"Hikkaduwa Beach, Sri Lanka",4.7,1438.0,['Hikkaduwa Beach is a delightful escape for s...,'Hikkaduwa Beach is a delightful escape for so...
...,...,...,...,...,...,...,...,...,...
393,404,Nilaveli Beach,8.700307,81.192050,"Nilaveli Beach, Sri Lanka",4.5,1247.0,['Nilaveli Beach is a beautiful escape with st...,'Nilaveli Beach is a beautiful escape with stu...
394,406,Uppuveli Beach,8.607956,81.220013,"Trincomalee, Sri Lanka",4.3,399.0,['Uppuveli Beach is a stunning escape! The sof...,'Uppuveli Beach is a stunning escape! The soft...
395,407,Koggala Beach,5.992272,80.310691,"Koggala Beach, Sri Lanka",4.3,353.0,['Koggala Beach is a hidden gem! The soft sand...,'Koggala Beach is a hidden gem! The soft sand ...
396,408,Marakolliya Beach,6.042222,80.823073,"Kapuhenwala Road, Sri Lanka",4.3,180.0,['Marakolliya Beach is a hidden gem! The waves...,'Marakolliya Beach is a hidden gem! The waves ...


In [ ]:
places_df['formatted_address'] = places_df['formatted_address'].str.replace("Sri Lanka", "", regex=False)
places_df['formatted_address']

,formatted_address
0,"Arugam Bay Beach,"
1,"Mirissa,"
2,"Weligama,"
3,"Ahangama,"
4,"Hikkaduwa Beach,"
...,...
393,"Nilaveli Beach,"
394,"Trincomalee,"
395,"Koggala Beach,"
396,"Kapuhenwala Road,"


In [ ]:
places_df[places_df['formatted_address'].str.len() <= 4]

,place_id,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews,cleaned_reviews
11,11,Yala National Park,6.463961,81.471885,,4.10,2810.0,['Yala National Park is a true gem for wildlif...,'Yala National Park is a true gem for wildlife...
12,12,Udawalawe National Park,6.474629,80.876319,,4.30,6156.0,['Udawalawe National Park was absolutely breat...,'Udawalawe National Park was absolutely breath...
13,13,Wilpattu National Park,8.458182,80.051831,,4.70,2708.0,['Wilpattu National Park is a fantastic destin...,'Wilpattu National Park is a fantastic destina...
15,15,Minneriya National Park,8.015644,80.844670,,4.60,1293.0,['Minneriya National Park is a must-visit for ...,'Minneriya National Park is a mustvisit for wi...
16,16,Sinharaja Forest Reserve,6.406991,80.460027,,4.30,2062.0,['Exploring Sinharaja Forest Reserve was a mag...,'Exploring Sinharaja Forest Reserve was a magi...
17,17,Horton Plains National Park,6.809446,80.802333,,4.70,8564.0,"[\n""Horton Plains National Park was an incredi...",\nHorton Plains National Park was an incredibl...
89,89,Leisure World,6.921600,80.105600,,NaN,NaN,"['Leisure World has potential, but my experien...","'Leisure World has potential, but my experienc..."
120,120,Bambarakiri Ella,7.495246,80.699001,,4.40,2004.0,"['Bambarakiri Ella is a beautiful, serene spot...","'Bambarakiri Ella is a beautiful, serene spot ..."
124,124,Madunagala Hot Water Spring,6.253630,80.981711,,4.55,2268.0,['Madunagala Hot Water Spring is a hidden gem!...,'Madunagala Hot Water Spring is a hidden gem! ...
173,173,Mayan Water Park,7.068894,79.902236,"-,",3.70,562.0,['Mayan Water Park was a decent outing for my ...,'Mayan Water Park was a decent outing for my f...


In [ ]:
places_df.loc[places_df['place_id'] == 160, 'formatted_address'] = ""
places_df.loc[places_df['place_id'] == 173, 'formatted_address'] = ""

In [ ]:
places_df.loc[places_df['formatted_address'].str.len() > 1, 'formatted_address'] = \
    places_df['formatted_address'].str.rstrip(', ')

In [ ]:
places_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   place_id            398 non-null    int64  
 1   name                398 non-null    object 
 2   lat                 398 non-null    float64
 3   lng                 398 non-null    float64
 4   formatted_address   398 non-null    object 
 5   rating              343 non-null    float64
 6   user_ratings_total  343 non-null    float64
 7   latest_reviews      398 non-null    object 
 8   cleaned_reviews     398 non-null    object 
dtypes: float64(4), int64(1), object(4)
memory usage: 28.1+ KB


In [ ]:
places_df_cleaned = places_df.drop(columns=['place_id', 'latest_reviews'])
places_df_cleaned.head()

,name,lat,lng,formatted_address,rating,user_ratings_total,cleaned_reviews
0,Arugam Bay Beach,6.840408,81.836848,Arugam Bay Beach,4.8,1591.0,'Arugam Bay Beach is a surfer's paradise! I sp...
1,Mirissa Beach,5.944703,80.459161,Mirissa,4.6,1748.0,'Mirissa Beach is truly a gem on Sri Lankas so...
2,Weligama Beach (surf and stay),5.972486,80.435714,Weligama,4.4,325.0,'Weligama Beach is a fantastic spot for both b...
3,Ahangama,5.973975,80.362159,Ahangama,NaN,NaN,'Ahangama was a bit disappointing for me as a ...
4,Hikkaduwa Beach,6.137727,80.099060,Hikkaduwa Beach,4.7,1438.0,'Hikkaduwa Beach is a delightful escape for so...


In [ ]:
places_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                398 non-null    object 
 1   lat                 398 non-null    float64
 2   lng                 398 non-null    float64
 3   formatted_address   398 non-null    object 
 4   rating              343 non-null    float64
 5   user_ratings_total  343 non-null    float64
 6   cleaned_reviews     398 non-null    object 
dtypes: float64(4), object(3)
memory usage: 21.9+ KB


In [ ]:
# download the places_df_cleaned as csv
places_df_cleaned.to_csv('places_df_cleaned.csv', index=False)

# preprocessing done for places_df
---

---
#visitors_df

In [ ]:
visitors_df.head()

,User ID,Name,Email,Preferred Activities,Bucket list destinations Sri Lanka
0,1,Jennifer Quinn,jennifer.quinn@example.com,"['cycling', 'historical monuments', 'village h...","['Polonnaruwa', 'Hatton', 'Anuradhapura', 'Ell..."
1,2,Emily Perry,emily.perry@example.com,"['butterfly watching', 'hot springs', 'wildlif...","['Madunagala Hot Water Spring', 'Wilpattu Nati..."
2,3,Danielle Mcbride,danielle.mcbride@example.com,"['sea cruises', 'themed parks', 'craft worksho...","['Mirissa Beach', 'Negombo Lagoon', 'Batadomba..."
3,4,Angelica Wilson,angelica.wilson@example.com,"['fishing', 'hot springs', 'sailing']","['Maha Oya Hot Water Springs', 'Colombo Port C..."
4,5,Laurie Powers,laurie.powers@example.com,"['history tours', 'sailing', 'literary tours']","['Negombo Lagoon', 'Colombo Port City', 'Galle..."


In [ ]:
visitors_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   User ID                             10000 non-null  int64 
 1   Name                                10000 non-null  object
 2   Email                               10000 non-null  object
 3   Preferred Activities                10000 non-null  object
 4   Bucket list destinations Sri Lanka  10000 non-null  object
dtypes: int64(1), object(4)
memory usage: 390.8+ KB


In [ ]:
visitors_df.describe()

,User ID
count,10000.00000
mean,5000.50000
std,2886.89568
min,1.00000
25%,2500.75000
50%,5000.50000
75%,7500.25000
max,10000.00000


---
### checking id user id column has unique values for each rows

In [ ]:
visitors_df['User ID'].nunique()

10000

### checking if name column has names less than 3 characters

In [ ]:
visitors_df[visitors_df['Name'].str.len() < 3]

,User ID,Name,Email,Preferred Activities,Bucket list destinations Sri Lanka


### checking if there are special characters in names

In [ ]:
# Define a regex pattern to match names with special characters
special_char_pattern = r'[^a-zA-Z0-9.\s]'

# Filter rows where the 'Name' column contains special characters
visitors_df[visitors_df['Name'].str.contains(special_char_pattern, regex=True)]

,User ID,Name,Email,Preferred Activities,Bucket list destinations Sri Lanka


### checking similar names

In [ ]:
# Find all rows where the 'Name' column is duplicated
duplicates = visitors_df[visitors_df.duplicated('Name', keep=False)]

# Sort the duplicates by 'Name' to group them together
duplicates_sorted = duplicates.sort_values(by='Name').reset_index(drop=True)

# Display the original names along with their duplicates
duplicates_sorted

,User ID,Name,Email,Preferred Activities,Bucket list destinations Sri Lanka
0,1010,Aaron Phillips,aaron.phillips@example.com,"['butterfly watching', 'sailing', 'elephant ri...","['Colombo Port City', 'Knuckles', 'Udawalawe',..."
1,3780,Aaron Phillips,aaron.phillips@example.com,"['local crafts', 'hot air ballooning', 'hiking']","[""Sri Pada / Adam's Peak"", 'Ambalangoda', 'Kan..."
2,2877,Aaron Walker,aaron.walker@example.com,"['boat safaris', 'yoga retreats', 'museum visi...","['Colombo National Museum', 'Ratnapura Gem Mus..."
3,7372,Aaron Walker,aaron.walker@example.com,"['scuba diving', 'yoga retreats', 'photography']","['Kalpitiya', 'Yala National Park', 'Sinharaja..."
4,6674,Alicia Jones,alicia.jones@example.com,"['horse shows', 'sailing', 'themed parks']","['Viharamahadevi Park', 'Nuwara Eliya', 'Colom..."
...,...,...,...,...,...
1048,3406,William Wilson,william.wilson@example.com,"['horse shows', 'butterfly watching', 'paddleb...","['Kandy', 'Unawatuna Beach', 'Bentota River', ..."
1049,5123,Zachary Jones,zachary.jones@example.com,"['beachfront dining', 'landscape photography',...","['Ella Gap', 'Galle', 'Sigiriya', 'Ahangama', ..."
1050,4366,Zachary Jones,zachary.jones@example.com,"['river cruises', 'literary tours', 'public ar...","['Madu River', 'Kandy', 'Bentota River', 'Mart..."
1051,5061,Zachary Martin,zachary.martin@example.com,"['rock climbing', 'arts and culture', 'safaris']","['National Gallery of Art', 'Nelung Arts Centr..."


#### merged those duplicates with the originals by combining preferred activities and bucket list destinations srilanka

In [ ]:
# Function to merge lists in two columns
def merge_lists(group):
    group['Preferred Activities'] = group['Preferred Activities'].apply(eval)  # convert strings to lists
    group['Bucket list destinations Sri Lanka'] = group['Bucket list destinations Sri Lanka'].apply(eval)  # convert strings to lists

    # Merge lists of Preferred Activities and Bucket list destinations Sri Lanka
    merged_activities = list(set().union(*group['Preferred Activities']))
    merged_destinations = list(set().union(*group['Bucket list destinations Sri Lanka']))

    # Keep the first row and update the merged columns
    merged_row = group.iloc[0]
    merged_row['Preferred Activities'] = merged_activities
    merged_row['Bucket list destinations Sri Lanka'] = merged_destinations
    return merged_row

merged_df = duplicates_sorted.groupby('Name').apply(merge_lists).reset_index(drop=True)

In [ ]:
merged_df.iloc[1, ]["Preferred Activities"]

['museum visits',
 'photography',
 'scuba diving',
 'yoga retreats',
 'boat safaris']

In [ ]:
df_no_duplicates = visitors_df.drop_duplicates('Name', keep=False)
visitors_df = pd.concat([df_no_duplicates, merged_df], ignore_index=True)
visitors_df = visitors_df.sort_values(by='User ID')
visitors_df

,User ID,Name,Email,Preferred Activities,Bucket list destinations Sri Lanka
0,1,Jennifer Quinn,jennifer.quinn@example.com,"['cycling', 'historical monuments', 'village h...","['Polonnaruwa', 'Hatton', 'Anuradhapura', 'Ell..."
1,2,Emily Perry,emily.perry@example.com,"['butterfly watching', 'hot springs', 'wildlif...","['Madunagala Hot Water Spring', 'Wilpattu Nati..."
2,3,Danielle Mcbride,danielle.mcbride@example.com,"['sea cruises', 'themed parks', 'craft worksho...","['Mirissa Beach', 'Negombo Lagoon', 'Batadomba..."
3,4,Angelica Wilson,angelica.wilson@example.com,"['fishing', 'hot springs', 'sailing']","['Maha Oya Hot Water Springs', 'Colombo Port C..."
4,5,Laurie Powers,laurie.powers@example.com,"['history tours', 'sailing', 'literary tours']","['Negombo Lagoon', 'Colombo Port City', 'Galle..."
...,...,...,...,...,...
8942,9996,Jonathan Hernandez,jonathan.hernandez@example.com,"['paddleboarding', 'river cruises', 'kayaking']","['Ahungalla', 'Bolgoda Lake', 'Unawatuna Beach..."
8943,9997,Cody Gallegos,cody.gallegos@example.com,"['theater', 'scuba diving', 'yoga retreats']","['Kalpitiya', 'Hikkaduwa Coral Sanctuary', 'Tr..."
8944,9998,Amy House,amy.house@example.com,"['sea cruises', 'zip-lining', 'outdoor adventu...","['Hikkaduwa Coral Sanctuary', 'Ella', 'Pigeon ..."
8945,9999,Leslie Aguilar,leslie.aguilar@example.com,"['cycling', 'amusement parks', 'paddleboarding']","['Ella', 'Hatton', 'Negambo', 'Colombo Port Ci..."


### checking if there are emails with wrong format

In [ ]:
def is_valid_email(email):
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return re.match(pattern, email) is not None

# Apply function to the 'Email' column and filter invalid emails
visitors_df[~visitors_df['Email'].apply(is_valid_email)]

,User ID,Name,Email,Preferred Activities,Bucket list destinations Sri Lanka


---
### removing special characters from preffered activities

In [ ]:
def process_activities(row):
    reviews = row['Preferred Activities']  # Access the reviews
    cleaned_reviews = [clean_text(review) for review in reviews]  # Clean each review
    combined_string = ''.join(cleaned_reviews)
    return combined_string
visitors_df['Cleaned Preferred Activities'] = visitors_df.apply(process_activities, axis=1)
visitors_df.head()

,User ID,Name,Email,Preferred Activities,Bucket list destinations Sri Lanka,Cleaned Preferred Activities
0,1,Jennifer Quinn,jennifer.quinn@example.com,"['cycling', 'historical monuments', 'village h...","['Polonnaruwa', 'Hatton', 'Anuradhapura', 'Ell...","'cycling', 'historical monuments', 'village ho..."
1,2,Emily Perry,emily.perry@example.com,"['butterfly watching', 'hot springs', 'wildlif...","['Madunagala Hot Water Spring', 'Wilpattu Nati...","'butterfly watching', 'hot springs', 'wildlife..."
2,3,Danielle Mcbride,danielle.mcbride@example.com,"['sea cruises', 'themed parks', 'craft worksho...","['Mirissa Beach', 'Negombo Lagoon', 'Batadomba...","'sea cruises', 'themed parks', 'craft workshops'"
3,4,Angelica Wilson,angelica.wilson@example.com,"['fishing', 'hot springs', 'sailing']","['Maha Oya Hot Water Springs', 'Colombo Port C...","'fishing', 'hot springs', 'sailing'"
4,5,Laurie Powers,laurie.powers@example.com,"['history tours', 'sailing', 'literary tours']","['Negombo Lagoon', 'Colombo Port City', 'Galle...","'history tours', 'sailing', 'literary tours'"


---
### removing special characters from Bucket list destinations Sri Lanka

In [ ]:
def process_bucket_list(row):
    reviews = row['Bucket list destinations Sri Lanka']  # Access the reviews
    cleaned_reviews = [clean_text(review) for review in reviews]  # Clean each review
    combined_string = ''.join(cleaned_reviews)
    return combined_string
visitors_df['Cleaned Bucket list destinations Sri Lanka'] = visitors_df.apply(process_bucket_list, axis=1)
visitors_df.head()

,User ID,Name,Email,Preferred Activities,Bucket list destinations Sri Lanka,Cleaned Preferred Activities,Cleaned Bucket list destinations Sri Lanka
0,1,Jennifer Quinn,jennifer.quinn@example.com,"['cycling', 'historical monuments', 'village h...","['Polonnaruwa', 'Hatton', 'Anuradhapura', 'Ell...","'cycling', 'historical monuments', 'village ho...","'Polonnaruwa', 'Hatton', 'Anuradhapura', 'Ella..."
1,2,Emily Perry,emily.perry@example.com,"['butterfly watching', 'hot springs', 'wildlif...","['Madunagala Hot Water Spring', 'Wilpattu Nati...","'butterfly watching', 'hot springs', 'wildlife...","'Madunagala Hot Water Spring', 'Wilpattu Natio..."
2,3,Danielle Mcbride,danielle.mcbride@example.com,"['sea cruises', 'themed parks', 'craft worksho...","['Mirissa Beach', 'Negombo Lagoon', 'Batadomba...","'sea cruises', 'themed parks', 'craft workshops'","'Mirissa Beach', 'Negombo Lagoon', 'Batadombal..."
3,4,Angelica Wilson,angelica.wilson@example.com,"['fishing', 'hot springs', 'sailing']","['Maha Oya Hot Water Springs', 'Colombo Port C...","'fishing', 'hot springs', 'sailing'","'Maha Oya Hot Water Springs', 'Colombo Port Ci..."
4,5,Laurie Powers,laurie.powers@example.com,"['history tours', 'sailing', 'literary tours']","['Negombo Lagoon', 'Colombo Port City', 'Galle...","'history tours', 'sailing', 'literary tours'","'Negombo Lagoon', 'Colombo Port City', 'Galle ..."


In [ ]:
# create a cleaned visitors_df dataframe without preffered activities and bucket list column
visitors_df_cleaned = visitors_df.drop(columns=['Preferred Activities', 'Bucket list destinations Sri Lanka'])
visitors_df_cleaned.head()

,User ID,Name,Email,Cleaned Preferred Activities,Cleaned Bucket list destinations Sri Lanka
0,1,Jennifer Quinn,jennifer.quinn@example.com,"'cycling', 'historical monuments', 'village ho...","'Polonnaruwa', 'Hatton', 'Anuradhapura', 'Ella..."
1,2,Emily Perry,emily.perry@example.com,"'butterfly watching', 'hot springs', 'wildlife...","'Madunagala Hot Water Spring', 'Wilpattu Natio..."
2,3,Danielle Mcbride,danielle.mcbride@example.com,"'sea cruises', 'themed parks', 'craft workshops'","'Mirissa Beach', 'Negombo Lagoon', 'Batadombal..."
3,4,Angelica Wilson,angelica.wilson@example.com,"'fishing', 'hot springs', 'sailing'","'Maha Oya Hot Water Springs', 'Colombo Port Ci..."
4,5,Laurie Powers,laurie.powers@example.com,"'history tours', 'sailing', 'literary tours'","'Negombo Lagoon', 'Colombo Port City', 'Galle ..."


In [ ]:
# output the visitors_df_cleaned
visitors_df_cleaned.to_csv('visitors_df_cleaned.csv', index=False)